<a id="top"></a> 
# Basic Visualizations and Descriptive Statistics of Spike Train Data

<div class="alert alert-block alert-info">
_**Synopsis**_ <br><br>

**Data:** Spontaneous spiking activity from a retinal neuron in culture, exposed to low-light and high-light environments.
<br>

**Goal:** Visualize spike trains, compute and interpret descriptive statistics, and build simple models of interspike interval distributions as a function of the ambient light level.
<br>

**Tools:** Raster plots, interspike interval histograms, firing rate, autocorrelograms, maximum likelihood estimation, Kolmogorov-Smirnov plots.
</div>

* [Introduction](#.)
* [Data analysis](#data-analysis)
    1. [Visual inspection](#visual-inspection)
    2. [Examining the Interspike Intervals](#isi)
    3. [Examining Binned Spike Increments](#bsi)
    4. [Computing autocorrelations for the Increments](#autocorrelations)
    5. [Computing Autocorrelations of the ISIs](#acISI)
* [Summary](#summary)
* [Appendix: Spike Count Mean and Variance for a Poisson Process](#appendix)

## Introduction

Neurons in the retina typically respond to patterns of light displayed over small sections of the visual field. However, when retinal neurons are grown in culture and held under constant light and environmental conditions, they will still spontaneously fire action potentials. In a fully functioning retina, this spontaneous activity is sometimes described as background firing activity, which is modulated as a function of visual stimuli. It is useful to understand the properties of this background activity in order to determine in future experiments how these firing properties are affected by specific stimuli.

### Case Study Data

A researcher examining the background firing properties of one of these neurons contacts you to discuss his data. He records the spiking activity in one of two states, with the room lights off (low ambient light levels) or with the room lights on (high ambient light levels). He would like to collaborate with you to determine whether there is a difference in background firing between these two conditions, and whether one environment is more conducive to future experimental analyses. He records the spiking activity for 30 seconds in each condition.

### Goal

Typically the first step in any data analysis involves visualizing and using simple descriptive statistics to characterize pertinent features of the data. For time series data that take on a continuous value at each time point, like the field potentials analyzed in earlier chapters, we typically start by simply plotting each data value as a function of time. For spike train data, things can become a bit more complicated. One reason for this is that there are multiple equivalent ways to describe the same spike train data. The data could be stored as a sequence of spike times; as a sequence of waiting times between spikes, or interspike intervals; or as a discrete time series indicating the number of spikes in discrete time bins. Knowing how to manipulate and visualize spike train data using all these different representations is the first step to understanding the structure present in the data and is the primary goal of this chapter.

### Tools

We develop tools in this chapter to visualize spike train data and to provide basic statistical methods appropriate for analyzing spike trains.

<a id="data-analysis"></a>
       
## Data Analysis

<a id="visual-inspection"></a>

Our data analysis begins with visual inspection. We load the ECoG data into Python and plot them by issuing the following commands: 

In [1]:
# Prepare the modules and plot settings
import scipy.io as sio
import matplotlib.pyplot as plt
import warnings
from matplotlib.pyplot import xlabel, ylabel, plot, show, title
from matplotlib import rcParams
%matplotlib inline

In [2]:
rcParams['figure.figsize'] = (12,3)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
data = sio.loadmat('Ch8-spikes-1.mat')  # Load the ECoG data

<div class="alert alert-block alert-info">

**Q.** How can we extract the variables of interest from `data`? Hint: Consider the `keys()` method.

</div>

You should find two non-private variables in the `data` dictionary:

`SpikesLow`: spike times over 30 s in the low ambient light condition
`SpikesHigh`: spike times over 30 s in the high ambient light condition

We can take these two variables from `data` so that we can work with them directly.

<div class="alert alert-block alert-warning">

Recall that the `loadmat()` function outputs a dictionary that contains the variables in the .mat file along with some additional information about the file. 

</div>

In [17]:
SpikesLow = data['SpikesLow'][0]
SpikesHigh = data['SpikesHigh'][0]

Each variable is a single vector that gives a set of increasing spike times for the associated condition. The two vectors are of different sizes because the neuron fired a different number of spikes in each condition.

<div class="alert alert-block alert-info">

<p>
    
**Q.** What is the size of the vector `SpikesLow`?

</p>

<p>
    
**A.** To answer this in Python, we use the command

    `SpikesLow.shape`

Python returns the answer
    
    `(750,)`
    
which reveals that `SpikesLow` is a vector with 750 elements (i.e., an array with 750 rows and 1 column). We could also have used `len(SpikesLow)` here since we are working with a vector (1-D) rather than a multidimensional array. Our collaborator who collected the data told us that each row holds a single spike time, and we continue to consider the implications of this statement.

</p>

</div>

<div class="alert alert-block alert-info">

**Q.** What is the size of the vector `SpikesHigh`?

</div>

Inspection of the sizes of the vectors `SpikesLow` and `SpikesHigh` reveals an important fact: the neuron fires more in the high-light condition. To make this observation more concrete, let’s compute the firing rate ($f$), defined mathematically as

$$
f = \frac{n}{T},
\tag{1}
$$

where $n$ is the number of spikes over the time interval $T$.

<div class="alert alert-block alert-info">

<p>
    
**Q.** What is the firing rate $f$ of the neuron recorded in the low ambient light condition?

</p>

<p>
    
**A.** To answer this question, we must first define two quantities of interest: $n$ and $T$. We consider here the entire duration of the recording, so $T = 30$ (our collaborator recorded the spiking activity for 30 s in each condition). During this interval, we found that the vector `SpikesLow` contains 750 spikes. With these two pieces of information, we may compute the firing rate.

</p>

</div>

In [29]:
T = 30
n = len(SpikesLow)
f = n / T

print('f =', f)

f = 25.0


This tells us that the firing rate is 25 spikes per second, or 25 Hz.

<div class="alert alert-block alert-info">

**Q.** Is this single number a good representation of the spike train data? What if the spiking changes dramatically throughout the recording?

</div>

<div class="alert alert-block alert-info">

**Q.** What is the firing rate of the neuron recorded in the high ambient light condition? 

</div>

In [ ]:
from IPython.core.display import HTML
HTML('../style.css')